In [1]:
import sys, os
if os.path.abspath(os.pardir) not in sys.path:
    sys.path.insert(0, os.path.abspath(os.pardir))
import CONFIG
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import preprocessing
from sklearn import model_selection
from tqdm.autonotebook import tqdm

In [37]:
DATA_DIR = CONFIG.CFG.DATA.BASE
K_FOLDS = 5
BATCH_SIZE = 32
LEARNING_RATE = 5e-4
NUM_EPOCHS = 1100
# print loss every
PRINT_EVERY = 50

QUANTILES = [0.3, 0.5, 0.7]
SCALE_COLUMNS = ['Weeks', 'FVC', 'Percent', 'Age'] #'Percent'
SCALE_COLUMNS = ['Weeks_Passed', 'Base_FVC', 'Base_Percent', 'Base_Age']
SEX_COLUMNS = ['Male', 'Female']
SMOKING_STATUS_COLUMNS = ['Currently smokes', 'Ex-smoker', 'Never smoked']
FV = SEX_COLUMNS + SMOKING_STATUS_COLUMNS + SCALE_COLUMNS
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [38]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(2020)

In [39]:
kf = model_selection.KFold(K_FOLDS)
MIN_MAX_SCALER = preprocessing.MinMaxScaler()

In [40]:
train_df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
test_df = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))
sub_df = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))
# remove the duplicates from the train_df
train_df.drop_duplicates(keep=False, inplace=True, subset=['Patient', 'Weeks'])

In [41]:
# extract the Patient and weeks from the Patient_Week column
sub_df['Patient'] = sub_df['Patient_Week'].apply(lambda x: x.split('_')[0])
sub_df['Weeks'] = sub_df['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
sub_df.head()

,Patient_Week,FVC,Confidence,Patient,Weeks
0,ID00419637202311204720264_-12,2000,100,ID00419637202311204720264,-12
1,ID00421637202311550012437_-12,2000,100,ID00421637202311550012437,-12
2,ID00422637202311677017371_-12,2000,100,ID00422637202311677017371,-12
3,ID00423637202312137826377_-12,2000,100,ID00423637202312137826377,-12
4,ID00426637202313170790466_-12,2000,100,ID00426637202313170790466,-12


In [42]:
# merge the sub_df with the test_df
sub_df = sub_df.drop('FVC', axis=1).merge(test_df.drop('Weeks', axis=1), on='Patient')
sub_df.head()

,Patient_Week,Confidence,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00419637202311204720264_-12,100,ID00419637202311204720264,-12,3020,70.186855,73,Male,Ex-smoker
1,ID00419637202311204720264_-11,100,ID00419637202311204720264,-11,3020,70.186855,73,Male,Ex-smoker
2,ID00419637202311204720264_-10,100,ID00419637202311204720264,-10,3020,70.186855,73,Male,Ex-smoker
3,ID00419637202311204720264_-9,100,ID00419637202311204720264,-9,3020,70.186855,73,Male,Ex-smoker
4,ID00419637202311204720264_-8,100,ID00419637202311204720264,-8,3020,70.186855,73,Male,Ex-smoker


In [43]:
train_df['FROM'] = 'train'
test_df['FROM'] = 'val'
sub_df['FROM'] = 'test'

In [44]:
sub_df.shape

(730, 10)

In [45]:
test_df.shape

(5, 8)

In [46]:
combined_df = train_df.append([test_df, sub_df])

In [47]:
# initialize base_week column
combined_df['Base_Week'] = combined_df['Weeks']
# make the weeks from sub_df to be np.nan so that when we calculate the base_week it comes from the test_df
combined_df.loc[combined_df['FROM'] == 'test', 'Base_Week'] = np.nan
# now calculate the min for each patient group and set it to the Base_Week column
combined_df['Base_Week'] = combined_df.groupby('Patient')['Base_Week'].transform('min')

In [48]:
# get the base_df (where the Base_Week == the min_week we calculated) so that we can get the base_fvc, base_age and base_percentage
base_df = combined_df[combined_df['Weeks'] == combined_df['Base_Week']]

In [49]:
base_df.rename(columns={
    'FVC': 'Base_FVC',
    'Percent': 'Base_Percent',
    'Age': 'Base_Age'
}, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [50]:
combined_df = combined_df.merge(base_df[['Patient', 'Base_FVC', 'Base_Percent', 'Base_Age']], on='Patient', how='left')

In [51]:
combined_df.tail()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,FROM,Patient_Week,Confidence,Base_Week,Base_FVC,Base_Percent,Base_Age
3821,ID00426637202313170790466,132,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_132,100.0,0.0,2925,71.824968,73
3822,ID00426637202313170790466,132,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_132,100.0,0.0,2925,71.824968,73
3823,ID00426637202313170790466,133,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_133,100.0,0.0,2925,71.824968,73
3824,ID00426637202313170790466,133,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_133,100.0,0.0,2925,71.824968,73
3825,ID00426637202313170790466,133,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_133,100.0,0.0,2925,71.824968,73


In [52]:
combined_df['Weeks_Passed'] = combined_df['Weeks'] - combined_df['Base_Week']

In [53]:
combined_df.tail()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,FROM,Patient_Week,Confidence,Base_Week,Base_FVC,Base_Percent,Base_Age,Weeks_Passed
3821,ID00426637202313170790466,132,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_132,100.0,0.0,2925,71.824968,73,132.0
3822,ID00426637202313170790466,132,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_132,100.0,0.0,2925,71.824968,73,132.0
3823,ID00426637202313170790466,133,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_133,100.0,0.0,2925,71.824968,73,133.0
3824,ID00426637202313170790466,133,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_133,100.0,0.0,2925,71.824968,73,133.0
3825,ID00426637202313170790466,133,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_133,100.0,0.0,2925,71.824968,73,133.0


In [54]:
MIN_MAX_SCALER.fit(combined_df[combined_df['FROM'] == 'train'][['Weeks_Passed', 'FVC', 'Percent', 'Age']])

MinMaxScaler(copy=True, feature_range=(0, 1))

In [55]:
combined_df.tail()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,FROM,Patient_Week,Confidence,Base_Week,Base_FVC,Base_Percent,Base_Age,Weeks_Passed
3821,ID00426637202313170790466,132,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_132,100.0,0.0,2925,71.824968,73,132.0
3822,ID00426637202313170790466,132,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_132,100.0,0.0,2925,71.824968,73,132.0
3823,ID00426637202313170790466,133,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_133,100.0,0.0,2925,71.824968,73,133.0
3824,ID00426637202313170790466,133,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_133,100.0,0.0,2925,71.824968,73,133.0
3825,ID00426637202313170790466,133,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_133,100.0,0.0,2925,71.824968,73,133.0


In [56]:
combined_df[['Weeks_Passed', 'Base_FVC', 'Base_Percent', 'Base_Age']] = MIN_MAX_SCALER.transform(combined_df[['Weeks_Passed', 'Base_FVC', 'Base_Percent', 'Base_Age']])

In [57]:
# convert categoricals into dummies
combined_df['Sex'] = pd.Categorical(combined_df['Sex'], categories=SEX_COLUMNS)
combined_df['SmokingStatus'] = pd.Categorical(combined_df['SmokingStatus'], categories=SMOKING_STATUS_COLUMNS)
combined_df = combined_df.join(pd.get_dummies(combined_df['Sex']))
combined_df = combined_df.join(pd.get_dummies(combined_df['SmokingStatus']))

In [58]:
combined_df.drop_duplicates(inplace=True)

In [59]:
combined_df.reset_index(drop=True)

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,FROM,Patient_Week,Confidence,Base_Week,Base_FVC,Base_Percent,Base_Age,Weeks_Passed,Male,Female,Currently smokes,Ex-smoker,Never smoked
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,train,NaN,NaN,-4.0,0.267050,0.236393,0.769231,0.000000,1,0,0,1,0
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker,train,NaN,NaN,-4.0,0.267050,0.236393,0.769231,0.142857,1,0,0,1,0
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker,train,NaN,NaN,-4.0,0.267050,0.236393,0.769231,0.174603,1,0,0,1,0
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker,train,NaN,NaN,-4.0,0.267050,0.236393,0.769231,0.206349,1,0,0,1,0
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker,train,NaN,NaN,-4.0,0.267050,0.236393,0.769231,0.238095,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2265,ID00426637202313170790466,129,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_129,100.0,0.0,0.376525,0.345604,0.615385,2.047619,1,0,0,0,1
2266,ID00426637202313170790466,130,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_130,100.0,0.0,0.376525,0.345604,0.615385,2.063492,1,0,0,0,1
2267,ID00426637202313170790466,131,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_131,100.0,0.0,0.376525,0.345604,0.615385,2.079365,1,0,0,0,1
2268,ID00426637202313170790466,132,2925,71.824968,73,Male,Never smoked,test,ID00426637202313170790466_132,100.0,0.0,0.376525,0.345604,0.615385,2.095238,1,0,0,0,1


In [60]:
class PulmonaryDataset(torch.utils.data.Dataset):
    def __init__(self, df, FV, test=False):
        self.df = df
        self.test = test
        self.FV = FV

    def __getitem__(self, idx):
        return {
            'features': torch.tensor(self.df[self.FV].iloc[idx].values),
            'target': torch.tensor(self.df['FVC'].iloc[idx])
        }

    def __len__(self):
        return len(self.df)

In [61]:
class PulmonaryModel(nn.Module):
    def __init__(self, in_features=9, out_quantiles=3):
        super(PulmonaryModel, self).__init__()
        self.fc1 = nn.Linear(in_features, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, out_quantiles)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [62]:
clip_one, clip_two = torch.tensor(70, dtype=torch.float32).to(DEVICE), torch.tensor(1000, dtype=torch.float32).to(DEVICE)
def score(y_true, y_pred):
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]

    sigma_clip = torch.max(sigma, clip_one)
    delta = torch.abs(y_true - fvc_pred)
    delta = torch.min(delta, clip_two)
    sq2 = torch.sqrt(torch.tensor(2, dtype=torch.float32))
    metric = (delta / sigma_clip) * sq2 + torch.log(sigma_clip * sq2)
    return torch.mean(metric)

In [63]:
def quantile_loss(preds, target, quantiles, _lambda):
    assert not target.requires_grad
    assert preds.size(0) == target.size(0)
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - preds[:, i]
        losses.append(torch.max((q - 1) * errors, q * errors).unsqueeze(1))
    loss = torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))
    # return loss
    return _lambda * loss + (1 - _lambda) * score(target, preds)

In [64]:
class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [65]:
def train_one_epoch(model, train_data_loader, optimizer, train_loss):
    model.train()
    for i, data in enumerate(train_data_loader):
        features = data['features']
        targets = data['target']

        features = features.to(DEVICE).float()
        targets = targets.to(DEVICE).float()

        model.zero_grad()
        out = model(features)
        loss = quantile_loss(out, targets, QUANTILES, 0.6)
        train_loss.update(loss, features.size(0))
        loss.backward()
        optimizer.step()

In [66]:
def eval_one_epoch(model, valid_data_loader, valid_loss, lr_scheduler):
    model.eval()

    with torch.no_grad():
        for i, data in enumerate(valid_data_loader):
            features = data['features']
            targets = data['target']

            features = features.to(DEVICE).float()
            targets = targets.to(DEVICE).float()
            
            out = model(features)
            loss = quantile_loss(out, targets, QUANTILES, 0.6)
            valid_loss.update(loss, features.size(0))
    
    if lr_scheduler is not None:
        lr_scheduler.step(valid_loss.avg)

In [67]:
new_train_df = combined_df[combined_df['FROM'] == 'train'].reset_index(drop=True)
TRAIN_PATIENTS = new_train_df['Patient'].unique().tolist()

In [68]:
for fold, (train_index, test_index) in enumerate(kf.split(TRAIN_PATIENTS)):
    model = PulmonaryModel(len(FV))
    model = model.to(DEVICE)

    df_train = new_train_df.iloc[train_index].reset_index(drop=True)
    df_valid = new_train_df.iloc[test_index].reset_index(drop=True)

    train_dataset = PulmonaryDataset(df_train, FV)
    valid_dataset = PulmonaryDataset(df_valid, FV)

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=10,
        shuffle=True,
        num_workers=4
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=4,
        shuffle=False,
        num_workers=4
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=50, factor=0.7, verbose=True)

    best_valid_loss = float('inf')

    
    # tq = tqdm(range(NUM_EPOCHS), desc=f"Fold {fold}")
    for epoch in range(NUM_EPOCHS):
        train_loss = AverageMeter()
        valid_loss = AverageMeter()

        train_one_epoch(model, train_data_loader, optimizer, train_loss)
        eval_one_epoch(model, valid_data_loader, valid_loss, lr_scheduler)

        if epoch % PRINT_EVERY == 0:
            print(f"Epoch {epoch}/{NUM_EPOCHS}, Loss {train_loss.avg}")
            print(f"Fold {fold}, Valid Loss {valid_loss.avg} \n")
        
        # tq.set_postfix(val_loss=valid_loss.avg.item())

        if valid_loss.avg < best_valid_loss:
            best_valid_loss = valid_loss.avg
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, os.path.join(CONFIG.CFG.DATA.MODELS_OUT, f"model_fold_{fold}.pt"))

Epoch 0/1100, Loss 2565.88232421875
Fold 0, Valid Loss 2740.00341796875 

Epoch 50/1100, Loss 444.3134765625
Fold 0, Valid Loss 444.6546936035156 

Epoch 100/1100, Loss 278.2435302734375
Fold 0, Valid Loss 367.0042724609375 

Epoch 150/1100, Loss 211.2231903076172
Fold 0, Valid Loss 313.4414978027344 

Epoch 200/1100, Loss 178.32240295410156
Fold 0, Valid Loss 281.9181213378906 

Epoch 250/1100, Loss 160.99185180664062
Fold 0, Valid Loss 262.3938903808594 

Epoch 300/1100, Loss 152.3524627685547
Fold 0, Valid Loss 248.6693572998047 

Epoch 350/1100, Loss 146.09017944335938
Fold 0, Valid Loss 237.56304931640625 

Epoch 400/1100, Loss 140.57394409179688
Fold 0, Valid Loss 229.41932678222656 

Epoch 450/1100, Loss 136.79217529296875
Fold 0, Valid Loss 225.1420440673828 

Epoch 500/1100, Loss 133.68885803222656
Fold 0, Valid Loss 221.9834747314453 

Epoch 550/1100, Loss 130.76727294921875
Fold 0, Valid Loss 217.43038940429688 

Epoch 600/1100, Loss 127.92623138427734
Fold 0, Valid Loss 213

In [71]:
CONFIG.upload_to_kaggle("osicqrmodel", "OSIC QR Model", new=False)

In [70]:
models = []
for fold in range(K_FOLDS):
    model = PulmonaryModel(len(FV))
    model = model.to(DEVICE)
    checkpoint = torch.load(os.path.join(CONFIG.CFG.DATA.MODELS_OUT, f"model_fold_{fold}.pt"))
    model.load_state_dict(checkpoint['model_state_dict'])
    models.append(model)

In [134]:
new_test_df = combined_df[combined_df['FROM'] == 'test'].reset_index(drop=True)
test_dataset = PulmonaryDataset(new_test_df, FV)
test_data_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=4
)

In [135]:
new_test_df.shape

(730, 20)

In [136]:
avg_preds = np.zeros((len(test_dataset), len(QUANTILES)))
with torch.no_grad():
    for model in models:
        preds = []
        for j, test_data in enumerate(test_data_loader):
            features = test_data['features']
            targets = test_data['target']

            features = features.to(DEVICE).float()
            targets = targets.to(DEVICE).float()

            out = model(features)
            preds.append(out)
        preds = torch.cat(preds, dim=0).cpu().numpy()
        avg_preds += preds
avg_preds /= len(models)

In [141]:
avg_preds

array([[2898.08457031, 3142.53564453, 3377.50253906],
       [2892.51005859, 3136.5050293 , 3371.03310547],
       [2886.93530273, 3130.47456055, 3364.56391602],
       ...,
       [2028.98952637, 2202.49018555, 2369.36118164],
       [2023.7012207 , 2196.76708984, 2363.21899414],
       [2018.41291504, 2191.04406738, 2357.07675781]])

In [57]:
sub_df.shape

(730, 10)

In [138]:
temp_df = new_test_df

In [139]:
temp_df['FVC'] = avg_preds[:, 1]
temp_df['Confidence'] = np.abs(avg_preds[:, 2] - avg_preds[:, 0])

In [140]:
temp_df.to_csv("view.csv", index=False)

In [48]:
# # inverse the scaling operation for FVC
# avg_preds -= MIN_MAX_SCALER.min_[SCALE_COLUMNS.index('FVC')]
# avg_preds /= MIN_MAX_SCALER.scale_[SCALE_COLUMNS.index('FVC')]